In [1]:
import itertools
import os

In [2]:
bash_scripts_dir = '../local_bash_scripts/June10_3sites/'

In [15]:
site_labels = ['GCN', 'FR', 'EC']

In [16]:
param_dicts = {}

In [17]:
# June 10
# site	pgerm	fmean	fvar	wA	    wB
# FR	0.165	1.306	1.403	1.453	0.547
# EC	0.224	2.858	23.513	1.848	0.152
# GCN	0.227	2.097	13.746	1.884	0.116

In [6]:
param_dicts['FR'] = {
    'all_fits': [1],
    'all_envs': [1],
    'all_germs': [1],
    'all_hards': [1, 0],
    'all_pA': [0, 0.25, 0.5, 0.75, 1],
    'all_pGermBH': [0.165],
    'all_pGermWT': [0.165, 0.5, 0.75, 1],
    'all_bankLength': [3, 5, 7],
    'all_fMean': [1.306],
    'all_fVar': [1.403],
    'all_wA': [1.453],
    'all_wB': [0.547],
    'site': ['FR']
}

In [7]:
param_dicts['EC'] = {
    'all_fits': [1],
    'all_envs': [1],
    'all_germs': [1],
    'all_hards': [1, 0],
    'all_pA': [0, 0.25, 0.5, 0.75, 1],
    'all_pGermBH': [0.224],
    'all_pGermWT': [0.224, 0.5, 0.75, 1],
    'all_bankLength': [3, 5, 7],
    'all_fMean': [2.858],
    'all_fVar': [23.513],
    'all_wA': [1.848],
    'all_wB': [0.152],
    'site': ['EC']
}

In [8]:
param_dicts['GCN'] = {
    'all_fits': [1],
    'all_envs': [1],
    'all_germs': [1],
    'all_hards': [1, 0],
    'all_pA': [0, 0.25, 0.5, 0.75, 1],
    'all_pGermBH': [0.227],
    'all_pGermWT': [0.227, 0.5, 0.75, 1],
    'all_bankLength': [3, 5, 7],
    'all_fMean': [2.097],
    'all_fVar': [13.746],
    'all_wA': [1.884],
    'all_wB': [0.116],
    'site': ['GCN']
}

In [9]:
'''
returns a list of dicts where each dict is a combination of parameters
'''
def generate_param_combinations(param_dict):
    keys = param_dict.keys()
    values = (param_dict[key] for key in keys)
    return [dict(zip(keys, combination)) for combination in itertools.product(*values)]

In [10]:
index = 0
for site in site_labels:
    site_dict = param_dicts[site]

    # ab sim
    ab_dicts = {k: v for k, v in site_dict.items() if k not in ['all_fMean', 'all_fVar']}
    ab_combination_dicts = generate_param_combinations(ab_dicts)

    for c in ab_combination_dicts:
        etype = c['all_envs']
        gtype = c['all_germs']
        stype = c['all_hards']
        pA = c['all_pA']
        pGermBH = c['all_pGermBH']
        pGermWT = c['all_pGermWT']
        bankLength = c['all_bankLength']
        wA = c['all_wA']
        wB = c['all_wB']
        site = c['site']

        ab_command = 'julia julia_scripts/delayedgerm_sim.jl {etype} {gtype} {stype} {pA} {pGermBH} {pGermWT} {bankLength} {wA} {wB} {site}'.format(
            etype=etype,
            gtype=gtype,
            stype=stype,
            pA=pA,
            pGermBH=pGermBH,
            pGermWT=pGermWT,
            bankLength=bankLength,
            wA=wA,
            wB=wB,
            site=site
        )

        index += 1

        # create a bash script for each command
        with open(bash_scripts_dir + str(index) + '.sh', 'w') as f:
            f.write(ab_command)

    # dist sim
    dist_dicts = {k: v for k, v in site_dict.items() if k not in ['all_pA', 'all_wA', 'all_wB']}
    dist_combination_dicts = generate_param_combinations(dist_dicts)

    for c in dist_combination_dicts:
        ftype = c['all_fits']
        gtype = c['all_germs']
        stype = c['all_hards']
        pGermBH = c['all_pGermBH']
        pGermWT = c['all_pGermWT']
        bankLength = c['all_bankLength']
        fMean = c['all_fMean']
        fVar = c['all_fVar']
        site = c['site']

        dist_command = 'julia julia_scripts/delayedgerm_sim_dist.jl {ftype} {gtype} {stype} {pGermBH} {pGermWT} {bankLength} {fMean} {fVar} {site}'.format(
            ftype=ftype,
            gtype=gtype,
            stype=stype,
            pGermBH=pGermBH,
            pGermWT=pGermWT,
            bankLength=bankLength,
            fMean=fMean,
            fVar=fVar,
            site=site
        )

        index += 1

        # create a bash script for each command
        with open(bash_scripts_dir + str(index) + '.sh', 'w') as f:
            f.write(dist_command)

In [11]:
# create a slurm script to run all the bash scripts
slurm_job_name = 'june10_3sites'
slurm_script_path = f'../slurm_scripts/{slurm_job_name}.sh'
slurm_job_size = index

In [12]:

slurm_script = """#!/bin/bash

#SBATCH -J {slurm_job_name}           # Job name.

#SBATCH -N 1                               # Number of cores.
#SBATCH -t 48:00:00                         # Runtime in D-HH:MM (or use minutes).
#SBATCH --mem 4g                           # Memory in GB.
#SBATCH --array=1-{slurm_job_size}                    # Array range.
#SBATCH --mail-type=ALL                    # Type of email notification: BEGIN,END,FAIL,ALL.
#SBATCH --mail-user=zheng_yin@alumni.brown.edu   # Email where notifications will be sent.
#SBATCH -o {slurm_job_name}-%A.out
#SBATCH -e {slurm_job_name}-%A.err


# set script dir
bash_scripts_dir="local_bash_scripts/June10_3sites"

# Get all the bash scripts in the dir
bash_scripts=($(ls $bash_scripts_dir))

# Get job index
job_index=$((SLURM_ARRAY_TASK_ID))

# Run each bash script
script=${{bash_scripts_dir/$bash_scripts[$job_index]}}
# ./$script
echo "Running $script"

"""

In [13]:
slurm_script = slurm_script.format(
    slurm_job_name=slurm_job_name,
    slurm_job_size=slurm_job_size,
    bash_scripts_dir=bash_scripts_dir
)

In [14]:
# Write the job script to a file
with open(slurm_script_path, 'w') as f:
    f.write(slurm_script)